# Fine-tune EoMT with LoRA on Google Colab

This notebook provides a step-by-step guide to fine-tune the EoMT (End-of-Model-Training) model using Logit Normalization and LoRA (Low-Rank Adaptation). 

## 1. Environment Setup
First, we check the GPU status and install necessary dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

### Mount Google Drive
Run the cell below to mount your Google Drive. You will be asked to authorize access.
After mounting, your Drive files will be available at `/content/drive/MyDrive`.

**IMPORTANT:** Update `PROJECT_PATH` below to the folder in your Drive where you uploaded this project.

In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define your project path (Change this to your actual folder name)
PROJECT_PATH = '/content/drive/MyDrive/MaskArchitectureAnomaly_CourseProject'

# 3. Change directory
try:
    os.chdir(PROJECT_PATH)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The path '{PROJECT_PATH}' does not exist. Please check the path variable.")

## 2. Install Dependencies
We need to install the libraries specified in `eomt/requirements.txt`.

In [ ]:
!pip install -r eomt/requirements.txt

## 3. Python Path Setup
To ensure all modules (like `training`, `eomt`, `datasets`) are importable, we add the necessary paths to `sys.path`.

In [ ]:
import sys
import os

# Add current directory (project root)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Add 'eomt' subdirectory as it contains 'training' and 'datasets' packages
eomt_path = os.path.join(os.getcwd(), 'eomt')
if eomt_path not in sys.path:
    sys.path.append(eomt_path)

print("Python path updated.")

## 4. Run Training with LoRA
Verified training command using `train_net_lora.py` with PyTorch Lightning CLI.

### Key Arguments:
- `--lora_rank`: Rank of the LoRA adapters (default: 32).
- `--logit_norm_temperature`: Temperature for logit normalization (default: 0.04).
- `--head_only`: If set, only trains the head (faster, less memory).
- `--trainer.default_root_dir`: Directory to save checkpoints and logs.
- `--config`: Path to the YAML config file.

**Note:** Ensure your dataset path in the config file or command argument points to the correct location in Colab.

In [ ]:
# Example Training Command
# Adjust paths (pretrained_path, data path) as necessary for your Colab environment

!python eomt/train_net_lora.py fit \
    --config eomt/configs/dinov2/cityscapes/semantic/eomt_base_640.yaml \
    --compile_disabled \
    --model.ckpt_path "trained_eomt/eomt_cityscapes.bin" \
    --data.init_args.img_size="[1024, 1024]" \
    --lora_rank 32 \
    --lora_alpha 64 \
    --logit_norm_temperature 0.04 \
    --trainer.default_root_dir "training_output" \
    --trainer.max_epochs 10 \
    --trainer.check_val_every_n_epoch 10 \
    --data.init_args.batch_size 4 \
    --trainer.accumulate_grad_batches 4 \
    --lora_targets="["class_head", "mask_head"]" \
    --trainer.logger.init_args.name "no-lora-t=0.04-no_load" \
    --data.init_args.path "eomt/cityscapes"

### Output Location
Checkpoints will be saved in `training_output/lightning_logs/version_X/checkpoints/`.

### Troubleshooting
- **ModuleNotFoundError**: Make sure you ran the `sys.path` cell and are in the correct root directory.
- **CUDA OOM**: Reduce `batch_size` or use `--head_only`.
- **Data Not Found**: verify `--data.init_args.path` is correct.

In [ ]:
!python eomt/finetune.py fit \
    --config eomt/configs/dinov2/cityscapes/semantic/eomt_base_640.yaml \
    --compile_disabled \
    --model.ckpt_path "trained_eomt/eomt_cityscapes.bin" \
    --trainer.max_epochs=10 \
    --trainer.check_val_every_n_epoch 10 \
    --data.init_args.img_size="[1024, 1024]" \
    --data.init_args.batch_size=4 \
    --trainer.accumulate_grad_batches 4 \
    --data.init_args.path="cityscapes" \
    --logit_norm_temperature 1.2 \
    --targets="["class_head", "mask_head"]" \
    --trainer.logger.init_args.name "no-lora-t=0.04-no_load" \
    --data.init_args.path "eomt/cityscapes"
